In [1]:
import sys
from pyprojroot import here
sys.path.insert(0, str(here()))

too complex to use two parameters, then here combine them into 1 gate (doge)

though it seems still complex

unfinished 023-0522

# Gate eigen-decoposition

In [8]:
%matplotlib inline
import numpy as np
import sympy as sp
from sympy import sqrt, symbols, pi, cos, Matrix, latex, trace
from IPython.display import display, Math

np.set_printoptions(precision=3, suppress=True)

#* H \otimes I
IDENTITY = np.array([[1, 0],
                     [0, 1]])

HADAMARD = np.array([[1/np.sqrt(2), 1/np.sqrt(2)],
                   [1/np.sqrt(2), -1/np.sqrt(2)]])

HI = np.kron(HADAMARD, IDENTITY)

CNOT = np.array([[1, 0, 0, 0],
                 [0, 1, 0, 0],
                 [0, 0, 0, 1],
                 [0, 0, 1, 0]])

eigenvalues, eigenvectors = np.linalg.eig(CNOT @ HI)

print("Eigenvalues:\n", eigenvalues)
print("Eigenvectors:\n", eigenvectors)

P = eigenvectors
D = np.diag(eigenvalues)
P_inv = np.linalg.inv(P)

print("P matrix:\n", P)
print("D matrix:\n", D)
print("P_inv matrix:\n", P_inv)

Eigenvalues:
 [-1.   +0.j     0.707+0.707j  0.707-0.707j  1.   +0.j   ]
Eigenvectors:
 [[ 0.271+0.j   0.   -0.5j  0.   +0.5j -0.653+0.j ]
 [ 0.271+0.j   0.   +0.5j  0.   -0.5j -0.653+0.j ]
 [-0.653+0.j   0.5  +0.j   0.5  -0.j  -0.271+0.j ]
 [-0.653+0.j  -0.5  -0.j  -0.5  +0.j  -0.271+0.j ]]
P matrix:
 [[ 0.271+0.j   0.   -0.5j  0.   +0.5j -0.653+0.j ]
 [ 0.271+0.j   0.   +0.5j  0.   -0.5j -0.653+0.j ]
 [-0.653+0.j   0.5  +0.j   0.5  -0.j  -0.271+0.j ]
 [-0.653+0.j  -0.5  -0.j  -0.5  +0.j  -0.271+0.j ]]
D matrix:
 [[-1.   +0.j     0.   +0.j     0.   +0.j     0.   +0.j   ]
 [ 0.   +0.j     0.707+0.707j  0.   +0.j     0.   +0.j   ]
 [ 0.   +0.j     0.   +0.j     0.707-0.707j  0.   +0.j   ]
 [ 0.   +0.j     0.   +0.j     0.   +0.j     1.   +0.j   ]]
P_inv matrix:
 [[ 0.271+0.j   0.271+0.j  -0.653+0.j  -0.653+0.j ]
 [-0.   +0.5j -0.   -0.5j  0.5  +0.j  -0.5  +0.j ]
 [-0.   -0.5j -0.   +0.5j  0.5  -0.j  -0.5  -0.j ]
 [-0.653-0.j  -0.653+0.j  -0.271-0.j  -0.271-0.j ]]


# Circuit Form U(θ)

In [16]:
# 定义符号变量
theta = sp.Symbol('theta', real=True)

# 定义矩阵D
D = sp.diag(sp.exp(sp.I*sp.pi*sp.cos(theta)**2), sp.exp(sp.I*sp.pi/4*sp.cos(theta)**2), sp.exp(-1*sp.I*sp.pi/4*sp.cos(theta)**2), 1)

display(Math(latex(D)))

U =  P * D * P_inv
U.simplify()
U = sp.expand_complex(U)
U = sp.nsimplify(U, tolerance=1e-10)
U = U.evalf(3)

print("U(θ):")
display(Math(latex(U)))

<IPython.core.display.Math object>

U(θ):


<IPython.core.display.Math object>

As the Z msmt result of input 00 and 11 are related to the front two columns of the matrix, we could easily found outputs of them are 1 and -1, respectively. Both of them are independent of θ.

# O_tilde

In [ ]:
from sympy import tensorproduct

Ob = np.array( [[ 1,  0,  0,  0],
                [ 0,  1,  0,  0],
                [ 0,  0,  1,  0],
                [ 0,  0,  0, -1]])

print("Observable matrix:")
print(Ob)

# 计算U(a)^\dagger O U(a)
U_dagger = U.H
O_tilde = U_dagger @ Ob @ U
O_tilde = O_tilde.simplify()

print("U:")
display(Math(latex(U)))
print("U(a)^\dagger:")
display(Math(latex(U_dagger)))
# print("U(a)^\dagger @ Ob:")
# display(Math(latex(U_dagger@Ob)))
print("O_tilde:")
display(Math(latex(O_tilde)))

# PO and SG

In [ ]:
from Symmetries.WernerSysSwapSymmetry import WernerSysSwapSymmetry
symmetry = WernerSysSwapSymmetry(1)

PO = symmetry._twirling(np.array(O_tilde.tolist()))
np.array(PO)

In [ ]:
SWAP = Matrix([[1, 0, 0, 0],
               [0, 0, 1, 0],
               [0, 1, 0, 0],
               [0, 0, 0, 1]])

S_dagger = SWAP.T  # SWAP 门是自它的转置的逆，因此 S^† = S^T
PO1 = S_dagger @ O_tilde @ SWAP
PO1.simplify()
print("PO1 matrix:")
display(Math(latex(PO1)))

II = Matrix([[1, 0, 0, 0],
               [0, 1, 0, 0],
               [0, 0, 1, 0],
               [0, 0, 0, 1]])
PO2 = II @ O_tilde @ II
PO2.simplify()
print("PO2 matrix:")
display(Math(latex(PO2)))

PO = 1/2 * (PO1 + PO2)

print("PO matrix:")
display(Math(latex(PO)))

# 计算 PO 和 O_tilde 之间的差
diff_matrix = PO - O_tilde

# 计算 H-S norm
# HS_norm = sqrt(sum([abs(elem)**2 for elem in diff_matrix]))
# HS_norm = sqrt(trace(diff_matrix.H * diff_matrix))
HS_norm = trace(diff_matrix.H * diff_matrix)

# 简化表达式
HS_norm = HS_norm.simplify()

print("HS_norm:")
display(Math(latex(HS_norm)))

we could see that, for all kinds of possible circuit parameter settings, twirling operation would make the prediction better or worse or keeps.

Then its hard to tell what would twirling do to the circuit.

# SG Plot

output on 00 and 11 would always be constant values as stated below, here we plot the SG relative to $\theta$

In [ ]:
import numpy as np
import sympy as sp
import matplotlib.pyplot as plt

ticks = np.linspace(-np.pi, np.pi, 9) 
tick_labels = [r'$-\pi$', r'$-\frac{3\pi}{4}$', r'$-\frac{\pi}{2}$', r'$-\frac{\pi}{4}$', 0, r'$\frac{\pi}{4}$', r'$\frac{\pi}{2}$', r'$\frac{3\pi}{4}$', r'$\pi$']

theta_samples = np.linspace(-np.pi, np.pi, 100)
phi_samples = np.linspace(-np.pi, np.pi, 100)
theta_samples, phi_samples = np.meshgrid(theta_samples, phi_samples)

y_sg_func = sp.lambdify([theta, phi], HS_norm, 'numpy')
y_sg_values = np.abs(y_sg_func(theta_samples, phi_samples))

fig = plt.figure(figsize=(12, 4))

# 3D 曲面子图
ax1 = fig.add_subplot(121, projection='3d')
ax1.plot_surface(theta_samples, phi_samples, y_sg_values, cmap='viridis')
ax1.set_xlabel(r'$\theta$')
ax1.set_ylabel(r'$\phi$')
ax1.set_xticks(ticks)
ax1.set_xticklabels(tick_labels)
ax1.set_yticks(ticks)
ax1.set_yticklabels(tick_labels)
ax1.set_zlabel(r'$\|\|H-S\|\|$')
ax1.set_title(r'3D Surface of $\|\|H-S\|\|$')

# Heatmap 子图
ax2 = fig.add_subplot(122)
c = ax2.pcolormesh(theta_samples, phi_samples, y_sg_values, cmap='viridis', shading='auto')
ax2.set_xlabel(r'$\theta$')
ax2.set_ylabel(r'$\phi$')
ax2.set_xticks(ticks)
ax2.set_xticklabels(tick_labels)
ax2.set_yticks(ticks)
ax2.set_yticklabels(tick_labels)
ax2.set_title(r'Heatmap of $\|\|H-S\|\|$')

# 添加 colorbar
fig.colorbar(c, ax=ax2)

plt.show()

In [ ]:
y_sg_func(np.pi/2, 0)

In [ ]:
# one test example

# po_func = sp.lambdify([theta, phi], PO, 'numpy')
# po_value = po_func(np.pi/2, 0)

# print("IX:\n",IX)
# O_ix = IX @ Ob @ IX
# print("U_ix:\n",O_ix)

# PO_ix1 = S_dagger @ O_ix @ SWAP
# print("PO_ix1:\n",PO_ix1)
# PO_ix2 = II @ O_ix @ II
# print("PO_ix2:\n",PO_ix2)
# PO_ix = 1/2 * (PO_ix1 + PO_ix2)
# print("PO_ix:\n",PO_ix)

# tmp_diff = PO_ix - O_ix
# print("tmp_diff:\n",tmp_diff)
# HS_norm_ix = trace(tmp_diff.H * tmp_diff)
# print("HS_norm_ix:\n",HS_norm_ix)

# Output forms of each input

In [ ]:
def zMsmtU_columnalue(U_column):
    odd_sum = sum([np.abs(U_column[i])**2 for i in range(1, len(U_column), 2)])
    even_sum = sum([np.abs(U_column[i])**2 for i in range(0, len(U_column), 2)])
    rs = sp.simplify(even_sum - odd_sum)
    return sp.nsimplify(rs, tolerance=1e-10)

display(Math(latex(U)))

zVals = [0, 0, 0, 0]

for i in range(4):
    U_column = U[:,i]
    display(Math(latex(U_column.T)))
    zVals[i] = zMsmtU_columnalue(U_column)
    display(Math(latex(zVals[i])))


# Summary
This is the same as the circuit $CNOT_{A \rightarrow B}^{(cos^2(\theta))}$, both the circuit output and SG

# loss function and label strategy

1. cc loss for each input
2. plot each 3-d view loss landscape

## square loss

In [ ]:
zLabels = [[1, -1, -1, 1], [-1, 1, 1, -1]] # y < 0 -> -1 as cat; y > 0 -> 1 as cat

theta_samples = np.linspace(-np.pi, np.pi, 100)
phi_samples = np.linspace(-np.pi, np.pi, 100)
theta_samples, phi_samples = np.meshgrid(theta_samples, phi_samples)

fig, axs = plt.subplots(4, 4, figsize=(20, 16))

for zIdx, zLabel in enumerate(zLabels):
    for i in range(4):
        y_square_loss = (zVals[i] - zLabel[i])**2
        y_square_loss_func = sp.lambdify([theta, phi], y_square_loss, 'numpy')
        y_square_loss_func_vectorized = np.vectorize(y_square_loss_func)
        y_square_loss_values = np.abs(y_square_loss_func_vectorized(theta_samples, phi_samples))

        axs[2*zIdx, i].set_title(f"Input state: {bin(i)} (Heatmap)")
        heatmap = axs[2*zIdx, i].imshow(y_square_loss_values, cmap='viridis', origin='lower',
                                extent=[-np.pi, np.pi, -np.pi, np.pi], aspect='auto')
        axs[2*zIdx, i].set_xlabel(r'$\theta$')
        axs[2*zIdx, i].set_ylabel(r'$\phi$')
        axs[2*zIdx, i].set_xticks(ticks)
        axs[2*zIdx, i].set_xticklabels(tick_labels)
        axs[2*zIdx, i].set_yticks(ticks)
        axs[2*zIdx, i].set_yticklabels(tick_labels)
        fig.colorbar(heatmap, ax=axs[2*zIdx, i])

        axs[2*zIdx+1, i] = fig.add_subplot(4, 4, 8*zIdx+i+5, projection='3d')
        axs[2*zIdx+1, i].set_title(f"Input state: {bin(i)} (3D plot)")
        surf = axs[2*zIdx+1, i].plot_surface(theta_samples, phi_samples, y_square_loss_values, cmap='viridis', edgecolor='none')
        axs[2*zIdx+1, i].set_xlabel(r'\theta')
        axs[2*zIdx+1, i].set_ylabel(r'$\phi$')
        axs[2*zIdx+1, i].set_xticks(ticks)
        axs[2*zIdx+1, i].set_xticklabels(tick_labels)
        axs[2*zIdx+1, i].set_yticks(ticks)
        axs[2*zIdx+1, i].set_yticklabels(tick_labels)
        axs[2*zIdx+1, i].set_zlabel('loss value')
    
plt.subplots_adjust(hspace=0.3)
plt.show()

then, add SG

In [ ]:
zLabels = [[1, -1, -1, 1], [-1, 1, 1, -1]] # y < 0 -> -1 as cat; y > 0 -> 1 as cat

theta_samples = np.linspace(-np.pi, np.pi, 100)
phi_samples = np.linspace(-np.pi, np.pi, 100)
theta_samples, phi_samples = np.meshgrid(theta_samples, phi_samples)

fig, axs = plt.subplots(4, 4, figsize=(20, 16))

ticks = np.linspace(-np.pi, np.pi, 9) 
tick_labels = [r'$-\pi$', r'$-\frac{3\pi}{4}$', r'$-\frac{\pi}{2}$', r'$-\frac{\pi}{4}$', 0, r'$\frac{\pi}{4}$', r'$\frac{\pi}{2}$', r'$\frac{3\pi}{4}$', r'$\pi$']

for zIdx, zLabel in enumerate(zLabels):
    for i in range(4):
        y_square_loss = (zVals[i] - zLabel[i])**2
        y_square_loss_func = sp.lambdify([theta, phi], y_square_loss, 'numpy')
        y_square_loss_func_vectorized = np.vectorize(y_square_loss_func)
        y_square_loss_values = np.abs(y_square_loss_func_vectorized(theta_samples, phi_samples))
        sg_values = y_sg_func(theta_samples, phi_samples)
        y_square_loss_values_sg = y_square_loss_values + sg_values

        axs[2*zIdx, i].set_title(f"Input state: {bin(i)} (Heatmap)")
        heatmap = axs[2*zIdx, i].imshow(y_square_loss_values_sg, cmap='viridis', origin='lower',
                                extent=[-np.pi, np.pi, -np.pi, np.pi], aspect='auto')
        axs[2*zIdx, i].set_xlabel(r'$\theta$')
        axs[2*zIdx, i].set_ylabel(r'$\phi$')
        axs[2*zIdx, i].set_xticks(ticks)
        axs[2*zIdx, i].set_xticklabels(tick_labels)
        axs[2*zIdx, i].set_yticks(ticks)
        axs[2*zIdx, i].set_yticklabels(tick_labels)
        fig.colorbar(heatmap, ax=axs[2*zIdx, i])

        axs[2*zIdx+1, i] = fig.add_subplot(4, 4, 8*zIdx+i+5, projection='3d')
        axs[2*zIdx+1, i].set_title(f"Input state: {bin(i)} (3D plot)")
        surf = axs[2*zIdx+1, i].plot_surface(theta_samples, phi_samples, y_square_loss_values_sg, cmap='viridis', edgecolor='none')
        axs[2*zIdx+1, i].set_xlabel(r'\theta')
        axs[2*zIdx+1, i].set_ylabel(r'$\phi$')
        axs[2*zIdx+1, i].set_xticks(ticks)
        axs[2*zIdx+1, i].set_xticklabels(tick_labels)
        axs[2*zIdx+1, i].set_yticks(ticks)
        axs[2*zIdx+1, i].set_yticklabels(tick_labels)
        axs[2*zIdx+1, i].set_zlabel('loss value')
    
plt.subplots_adjust(hspace=0.3)
plt.show()

this result seems good as SG do:

    1. provide small value in all candidate positions
    2. At the same time, loss curve provides two local minima points that one conserves symmetry and one not
    3. SG's local minima augments the depth of the symmetry conserved but sharpens another

TODO: But is this because in this case, the circuit is special that makes loss+SG suit for optimal points? Is there any case that loss+SG sharpens the optimal one but strengths others?

    1. SG candidates contains Best ones (1) + loss over-fitting ones under SG (2) + others (4)
    2. Loss candidates contains Best ones (1) + over-fitting ones with SG (2) + other over-fitting ones (3) (When model could be well-trained)
    3. Add SG would help Best ones (1) + over-fitting ones with under SG (2) + ... 
    4. (This is an unconvincing way)

    1. When model over-fits, it usually do not generalize well on even SG preserved samples.
    2. 

or say, there are some symmetry in QNNs?

SG角度：这里的SG给出了两种label策略下的解，从而在某个专门的loss下进行训练时，对应label策略下的解就会被加强，从而更为凸显。

loss角度：过拟合的解不符合SG的要求，从而被抵消，而同期出现的同水平的另外一组解被凸显。

## hinge loss

In [ ]:
# zLabels = [[1, -1, -1, 1], [-1, 1, 1, -1]] # y < 0 -> -1 as cat; y > 0 -> 1 as cat

# theta_samples = np.linspace(-np.pi, np.pi, 100)
# phi_samples = np.linspace(-np.pi, np.pi, 100)
# theta_samples, phi_samples = np.meshgrid(theta_samples, phi_samples)

# fig, axs = plt.subplots(4, 4, figsize=(20, 16))

# ticks = np.linspace(-np.pi, np.pi, 9) 
# tick_labels = [r'$-\pi$', r'$-\frac{3\pi}{4}$', r'$-\frac{\pi}{2}$', r'$-\frac{\pi}{4}$', 0, r'$\frac{\pi}{4}$', r'$\frac{\pi}{2}$', r'$\frac{3\pi}{4}$', r'$\pi$']

# for zIdx, zLabel in enumerate(zLabels):
#     for i in range(4):
#         y_hinge_loss = sp.Max(0, 1 - zVals[i] * zLabel[i])
#         y_hinge_loss_func = sp.lambdify([theta, phi], y_hinge_loss, 'numpy')
#         y_hinge_loss_func_vectorized = np.vectorize(y_hinge_loss_func)
#         y_hinge_loss_values = np.abs(y_hinge_loss_func_vectorized(theta_samples, phi_samples))

#         axs[2*zIdx, i].set_title(f"Input state: {bin(i)} (Heatmap)")
#         heatmap = axs[2*zIdx, i].imshow(y_hinge_loss_values, cmap='viridis', origin='lower',
#                                 extent=[-np.pi, np.pi, -np.pi, np.pi], aspect='auto')
#         axs[2*zIdx, i].set_xlabel(r'\theta')
#         axs[2*zIdx, i].set_ylabel(r'$\phi$')
#         axs[2*zIdx, i].set_xticks(ticks)
#         axs[2*zIdx, i].set_xticklabels(tick_labels)
#         axs[2*zIdx, i].set_yticks(ticks)
#         axs[2*zIdx, i].set_yticklabels(tick_labels)
#         fig.colorbar(heatmap, ax=axs[2*zIdx, i])

#         axs[2*zIdx+1, i] = fig.add_subplot(4, 4, 8*zIdx+i+5, projection='3d')
#         axs[2*zIdx+1, i].set_title(f"Input state: {bin(i)} (3D plot)")
#         surf = axs[2*zIdx+1, i].plot_surface(theta_samples, phi_samples, y_hinge_loss_values, cmap='viridis', edgecolor='none')
#         axs[2*zIdx+1, i].set_xlabel(r'\theta')
#         axs[2*zIdx+1, i].set_ylabel(r'$\phi$')
#         axs[2*zIdx+1, i].set_xticks(ticks)
#         axs[2*zIdx+1, i].set_xticklabels(tick_labels)
#         axs[2*zIdx+1, i].set_yticks(ticks)
#         axs[2*zIdx+1, i].set_yticklabels(tick_labels)
#         axs[2*zIdx+1, i].set_zlabel('loss value')
    
# plt.subplots_adjust(hspace=0.3)
# plt.show()